# Эксперименты с films доменом

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
from pathlib import Path
from tqdm import tqdm
import pickle
from time import time
from tqdm import tqdm
import pandas as pd
import numpy as np

import mlflow
import optuna

from src.models.BaseModel import TopRecommender
from src.models.ItemBasedRecommenders import CosineDistanceRecommender
from src.models.MatrixFactorizationRecommenders import ALSBasedRecommender, LightFMBasedRecommender

from src.metrics import average_single_precision

C:\Users\trybi\anaconda3\envs\IndexHockey\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [35]:
# Настраиваем MLFlow
mlflow.set_tracking_uri((Path.cwd() / '../src/models/mlflow_tracking/mlruns'))
mlflow.set_experiment('Films Domain')

<Experiment: artifact_location='file:///C:/Users/trybi/PycharmProjects/MFDP-RecSys/src/models/mlflow_tracking/mlruns/3', creation_time=1681633083831, experiment_id='3', last_update_time=1681633083831, lifecycle_stage='active', name='Films Domain', tags={}>

In [36]:
data_path = Path('../data')

In [37]:
df_interactions = pd.read_parquet(data_path / 'films_interactions.parquet')
df_films = pd.read_parquet(data_path / 'films_bd.parquet')

In [38]:
df_interactions = df_interactions.reset_index().drop('index', axis=1)

In [39]:
df_interactions

,user_id,item_id,rating
0,176549,f9506,-1
1,699317,f1659,1
2,1032142,f6686,1
3,1016458,f354,-1
4,884009,f693,-1
...,...,...,...
2397261,268216,f3071,1
2397262,438585,f7829,1
2397263,546862,f9673,-1
2397264,384202,f16197,1


In [40]:
# Отложим 1000 наблюдений в качестве тестовой выборки. На них будем считать метрику MAP@10.

test_samples = np.random.choice(range(len(df_interactions)), size = 1000, replace=False)

df_train = df_interactions.drop(test_samples)
df_test = df_interactions.iloc[test_samples]

In [41]:
# Соберем словарь {user: [items]}, потому что так быстрее будет работать поиск истории по юзерам
train_db = {user: items.item().split(', ') for user, items in (df_train
                                                               .groupby('user_id')
                                                               .agg({'item_id': lambda x: ', '.join(x)})
                                                               .iterrows())}

In [42]:
# Посмотрим максимальное количество посмотренных фильмов одним человеком.
max(len(i) for i in train_db.values())

350

In [43]:
# Соберем список пар (user: item) из test, чтобы по нему итерироваться
test_data = df_test[['user_id', 'item_id']].values

In [44]:
# Функция, которая будет считать эксперименты 
def experiment(run_name, model, model_params, recommend_params, my_favorites, k=10):
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(model_params)
        
        time_start = time()
        model.fit(interaction_data=df_train.rename({'user_id': 'user', 'item_id': 'item'}, axis=1),
                  **model_params)
        mlflow.log_metric(f'fit_time', time()-time_start)
        
        maps = []
        times = []
        for new_user, new_item in tqdm(test_data):
            user_favorites = train_db.get(new_user, [])
            time_start = time()
            recs = model.recommend(user_favorites, k, **recommend_params)
            times.append(time()-time_start)
            maps.append(average_single_precision(new_item, recs))
        
        
        mlflow.log_metric(f'MAP_on_{k}', sum(maps)/len(maps))
        mlflow.log_metric(f'time_to_rec', sum(times)/len(times))
        
        my_recs = model.recommend(my_favorites, k, **recommend_params)
        
        
        mlflow.log_text(' \n '.join(str(i) + ' - ' + str(j) + ' - ' + str(round(z)) + ' - ' + str(g) 
                                    for k in my_recs for i, j, z, g in df_films.query('item_id==@k')[['title',
                                                                                                      'directors',
                                                                                                      'release_year',
                                                                                                      'genres']].values),
                        'my_recs.txt')

        with open(model_path / f'{run_name}_model_series.pickle', 'wb') as f:
            pickle.dump(model, f)

        mlflow.log_artifact(model_path / f'{run_name}_model_series.pickle')

In [45]:
model_path = Path('../src/models/models_storage')

In [46]:
my_favorites = ['f8148', 'f14804', 'f11760', 'f3969', 'f1048', 'f2866', 'f4482', 'f11459', 'f3755', 'f10696']

In [47]:
df_films[df_films.item_id.isin(my_favorites)]

,item_id,title,release_year,genres,countries,age_rating,directors
107,f14804,Криминальное чтиво,1994.0,"зарубежные, триллеры, криминал, комедии",США,18.0,Квентин Тарантино
3123,f11459,Драйв,2011.0,"драмы, зарубежные, криминал",США,18.0,Николас Виндинг Рефн
8609,f4482,Господин никто,2009.0,"драмы, фантастика, фэнтези, мелодрамы","Германия, Бельгия",18.0,Жако Ван Дормель
9706,f3969,Пи,1997.0,"драмы, ужасы, фантастика, триллеры",США,16.0,Даррен Аронофски
9927,f11760,Реквием по мечте,2000.0,драмы,США,18.0,Даррен Аронофски
10785,f3755,Форрест Гамп,1994.0,"драмы, мелодрамы",США,12.0,Роберт Земекис
11420,f2866,"Карты, деньги, два ствола",1999.0,комедии,Великобритания,18.0,Гай Ричи
13279,f1048,Чёрный лебедь,2010.0,"драмы, триллеры",США,16.0,Даррен Аронофски
13803,f8148,Бешеные псы,1991.0,"драмы, триллеры",США,18.0,Квентин Тарантино
15296,f10696,Назад в будущее,1985.0,"фантастика, приключения, комедии",США,6.0,Роберт Земекис


In [55]:
experiment('Top count recommends', TopRecommender(), {'n_to_save':500, 'metric':'count'}, {'mode':'deterministic'},
           my_favorites, k=10)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3254.88it/s]


In [56]:
experiment('Random recommends', TopRecommender(), {'n_to_save':500, 'metric':'count'}, {'mode':'probabilistic'},
           my_favorites, k=10)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1785.71it/s]


In [58]:
experiment('CosineDistanceRecommender', CosineDistanceRecommender(), {}, {'mode':'deterministic'},
           my_favorites, k=10)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:26<00:00, 37.95it/s]


In [64]:
experiment('CosineDistanceRecommender + random', CosineDistanceRecommender(), {}, {'mode':'probabilistic'},
           my_favorites, k=10)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:27<00:00, 36.11it/s]


In [65]:
experiment('ALS Recommender', ALSBasedRecommender(), {}, {},
           my_favorites, k=10)

C:\Users\trybi\anaconda3\envs\IndexHockey\lib\site-packages\implicit\utils.py:34: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  "Intel MKL BLAS detected. Its highly recommend to set the environment "


  0%|          | 0/15 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 110.88it/s]


In [48]:
df_train['rating'] += 2

In [67]:
experiment('ALS Recommender + changed DS', ALSBasedRecommender(), {}, {},
           my_favorites, k=10)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 98.29it/s]


In [68]:
experiment('ALS Recommender + changed DS', ALSBasedRecommender(), {'factors': 10, 'regularization': 0.5, 'iterations': 30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 102.19it/s]


In [69]:
experiment('ALS Recommender + changed DS', ALSBasedRecommender(), {'factors': 50, 'regularization': 1, 'iterations': 30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 114.63it/s]


In [70]:
experiment('ALS Recommender + changed DS', ALSBasedRecommender(), {'factors': 20, 'regularization': 3, 'iterations': 50}, {},
           my_favorites, k=10)

  0%|          | 0/50 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 124.39it/s]


In [71]:
experiment('ALS Recommender + changed DS', ALSBasedRecommender(), {'factors': 200, 'regularization': 1, 'iterations': 50}, {},
           my_favorites, k=10)

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 63.87it/s]


### Подбор гипперпараметров

In [18]:
# Отложим 1000 наблюдений в качестве валидационной выборки

val_samples = np.random.choice(range(len(df_train)), size = 1000, replace=False)

df_train_val = df_train.iloc[val_samples]
df_train_train = df_train[~df_train.index.isin(df_train_val.index)]

In [19]:
train_db_new = {user: items.item().split(', ') for user, items in (df_train_train
                                                               .groupby('user_id')
                                                               .agg({'item_id': lambda x: ', '.join(x)})
                                                               .iterrows())}

In [20]:
# Напишем функцию для подбора гипперов
def objective(trial):
    factors = trial.suggest_int("factors", 5, 200)
    regularization = trial.suggest_float("regularization", 0, 20)
    
    model.fit(interaction_data=df_train_train.rename({'user_id': 'user', 'item_id': 'item'}, axis=1),
              factors = factors,
              regularization = regularization,
              iterations=30)
    
    maps = []
    for new_user, new_item in df_train_val[['user_id', 'item_id']].values:
        user_favorites = train_db_new.get(new_user, [])
        recs = model.recommend(user_favorites, 10)
        maps.append(average_single_precision(new_item, recs))

    
    return sum(maps)/len(maps)

In [21]:
model = ALSBasedRecommender()

In [22]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

[I 2023-04-18 13:21:02,147] A new study created in memory with name: no-name-13eec682-6dc4-49c1-ade2-557a382cd8af


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-04-18 13:23:58,403] Trial 0 finished with value: 0.00535535714285714 and parameters: {'factors': 98, 'regularization': 6.672605043737711}. Best is trial 0 with value: 0.00535535714285714.


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-04-18 13:26:37,229] Trial 1 finished with value: 0.0051353174603174555 and parameters: {'factors': 102, 'regularization': 3.778660492292516}. Best is trial 0 with value: 0.00535535714285714.


...

[I 2023-04-18 14:26:51,989] Trial 37 finished with value: 0.004769087301587299 and parameters: {'factors': 13, 'regularization': 3.640101597809995}. Best is trial 16 with value: 0.007771190476190463.


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-04-18 14:27:50,826] Trial 38 finished with value: 0.004839880952380949 and parameters: {'factors': 24, 'regularization': 9.04561607693111}. Best is trial 16 with value: 0.007771190476190463.


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-04-18 14:30:13,790] Trial 39 finished with value: 0.005313888888888883 and parameters: {'factors': 103, 'regularization': 5.209907133441384}. Best is trial 16 with value: 0.007771190476190463.


In [23]:
study.best_params

{'factors': 6, 'regularization': 10.704300633455523}

In [49]:
experiment('ALS Recommender + changed DS + optuna', ALSBasedRecommender(), {'factors': 6, 
                                                                   'regularization': 10.704300633455523, 
                                                                   'iterations': 30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 100.94it/s]
